In [1]:
import urllib.request, urllib.parse, urllib.error
import httplib2
from xml.dom import minidom

import getpass


In [6]:
def lim(lst, mx=10):
    return [x for x in lst][:mx]
def prt(events):
    lst = []
    for result in results.ResultsReader(events):
        lst.extend(result.values())
    return lst

In [3]:
# Main program to fetch Splunk sid, sessionKey

# from future import standard_library
# standard_library.install_aliases()

baseurl = "https://its824pl5spldp.svc.ny.gov:8089"
username = "bdk01"
password = getpass.getpass()

searchQuery = "search index=main sourcetype=cisco:ios | head 5"

# Authenticate with server.
# It is necessary to disable SSL cert validation. Splunk certs are self-signed.
# The followng will not work
# serverContent = httplib2.Http().request(baseurl + "/services/auth/login",

serverContent = httplib2.Http(disable_ssl_certificate_validation=True).request(baseurl + "/services/auth/login",
    "POST", headers={}, body=urllib.parse.urlencode({"username":username, "password":password}))[1]

sessionKey = minidom.parseString(serverContent).getElementsByTagName("sessionKey")[0].childNodes[0].nodeValue



········


In [4]:
print(f"{sessionKey=}")

sessionKey='Sv^yXbsWdplEo8TPh9Oi1DEJgO4ksd3VaMmUKM6i4GmA_XTKgphjTtOHQeHQvp^ddpw2sf^Akk148D^ioUguJhMd^phOC^DFoxFG12Wm_o'


In [8]:
# Here we are using the splunklib.client to login and show the apps loaded.

# My reading says that it should be possible to use environment variables 
# to store the password so it doesn;t have to be placed in the code. 
# My attempts in jupyter failed.


import splunklib.client as client
import sys
import os
import getpass

# sys.path.insert(0, os.path.join(os.path.dirname(__file__), "..", "lib"))
import splunklib.client as client

HOST = "its824pl5spldp.svc.ny.gov"
PORT = 8089
USERNAME = "bdk01"
PASSWORD = getpass.getpass()
OWNER = "bdk01"
APP = "search"

# Create a Service instance and log in 
service = client.connect(
    host=HOST,
    port=PORT,
    username=USERNAME,
    password=PASSWORD,
    owner=OWNER,
    app=APP)

# Print installed apps to the console to verify login
for app in lim(service.apps):
    print(app.name)


········
000_eventtypes
active_directory
alert_logevent
alert_test
alert_webhook
Antivirus
appsbrowser
aristanetworks
base64
biztalk


In [9]:
# List the saved searches that are available to the current user
# All the Reports can be seen because owner and app is specified with login.
# On this host the owner has role=admin so all are visible.

savedsearches = service.saved_searches

for savedsearch in lim(savedsearches):
    print(f"   {savedsearch.name=}")


   savedsearch.name='ADDM AI alert'
   savedsearch.name='ADDM FW alert'
   savedsearch.name='ADDM Hosts Alert'
   savedsearch.name='ADDM RT alert'
   savedsearch.name='ADDM SI alert'
   savedsearch.name='ADDM-jeff'
   savedsearch.name='AIX FS Over 90 Alert'
   savedsearch.name='Asset Usability'
   savedsearch.name='Assets by Agency'
   savedsearch.name='bdk01_test_report'


In [10]:
# Print the job history of saved searches
for savedsearch in service.saved_searches:
    if not savedsearch.name == 'License Usage Clone':
        continue
    print(savedsearch.name + ":")

    history = savedsearch.history()
    if len(history) > 0:
        for job in history:
            print(f"{job.name=}")
    else: 
        print("No jobs for this search")
    print

License Usage Clone:
No jobs for this search


In [11]:
from time import sleep
import sys

# Retrieve the new search
mysavedsearch = service.saved_searches['License Usage Clone']

# Run the saved search
job = mysavedsearch.dispatch()

# Create a small delay to allow time for the update between server and client
sleep(2)

# Wait for the job to finish--poll for completion and display stats
while True:
    job.refresh()
    stats = {"isDone": job["isDone"],
             "doneProgress": float(job["doneProgress"])*100,
              "scanCount": int(job["scanCount"]),
              "eventCount": int(job["eventCount"]),
              "resultCount": int(job["resultCount"])}
    status = ("\r%(doneProgress)03.1f%%   %(scanCount)d scanned   "
              "%(eventCount)d matched   %(resultCount)d results") % stats

    sys.stdout.write(status)
    sys.stdout.flush()
    if stats["isDone"] == "1":
        break
    sleep(2)

# Display the search results now that the job is done
jobresults = job.results()

while True:
    content = jobresults.read(1024)
    if len(content) == 0: break
    sys.stdout.write(content.decode(encoding='UTF-8'))
    sys.stdout.flush()
sys.stdout.write("\n")

100.0%   0 scanned   0 matched   0 results<results preview='0'/>


1

In [12]:
def prt(events):
    lst = []
    for result in results.ResultsReader(events):
        lst.extend(result.values())
    return lst

In [18]:
# This job makes a real search and outputs the result.
# Assume an authenticated session is open.

import sys
from time import sleep
import splunklib.results as results
import collections
# import ordereddict as odict

# ...
# 
# Initialize your service like so
# import splunklib.client as client
# service = client.connect(username="admin", password="yourpassword")

searchquery_normal = "search index=main | stats values(sourcetype)"

kwargs_normalsearch = {"earliest_time": "-15m",
                       "latest_time": "now",
                       "search_mode": "normal"}
job = service.jobs.create(searchquery_normal, **kwargs_normalsearch)

# A normal search returns the job's SID right away, so we need to poll for completion
while True:
    while not job.is_ready():
        pass
    stats = {"isDone": job["isDone"],
             "doneProgress": float(job["doneProgress"])*100,
              "scanCount": int(job["scanCount"]),
              "eventCount": int(job["eventCount"]),
              "resultCount": int(job["resultCount"])}

    status = ("\r%(doneProgress)03.1f%%   %(scanCount)d scanned   "
              "%(eventCount)d matched   %(resultCount)d results") % stats

    print(f"{status=}")
    sys.stdout.flush()
    if stats["isDone"] == "1":
        sys.stdout.write("\n\nDone!\n\n")
        break
    sleep(2)

# Get the results and display them
# for result in results.ResultsReader(job.results()):
#     print(result)

print(prt(job.results()))
print([x.decode() for x in list(job.results())])
    
job.cancel()   
# sys.stdout.write('\n')

status='\r0.0%   0 scanned   0 matched   0 results'
status='\r100.0%   23819 scanned   23819 matched   1 results'


Done!

[['ActiveDirectory', 'Perfmon:Available Memory', 'Perfmon:CPU Load', 'Perfmon:Free Disk Space', 'Perfmon:Network Interface', 'cisco:estreamer:log', 'cisco:estreamer:status', 'cisco:ios', 'git_update_corona', 'vmware:appvolumes']]
["<?xml version='1.0' encoding='UTF-8'?>\n", "<results preview='0'>\n", '<meta>\n', '<fieldOrder>\n', '<field>values(sourcetype)</field>\n', '</fieldOrder>\n', '</meta>\n', "\t<result offset='0'>\n", "\t\t<field k='values(sourcetype)'>\n", '\t\t\t<value><text>ActiveDirectory</text></value>\n', '\t\t\t<value><text>Perfmon:Available Memory</text></value>\n', '\t\t\t<value><text>Perfmon:CPU Load</text></value>\n', '\t\t\t<value><text>Perfmon:Free Disk Space</text></value>\n', '\t\t\t<value><text>Perfmon:Network Interface</text></value>\n', '\t\t\t<value><text>cisco:estreamer:log</text></value>\n', '\t\t\t<value><text>cisco:estreamer:status</t

In [46]:
# Reference:  https://www.twilio.com/blog/web-scraping-and-parsing-html-in-python-with-beautiful-soup

import requests
from bs4 import BeautifulSoup
html_text = ""
url = "https://scamalytics.com/ip/52.252.130.105"

# url = "https://live.euronext.com/fr/product/equities/FR0000120271-XPAR"
# html_text = requests.get(url, verify="/etc/ssl/certs/ca-certificates.crt").text
# html_text = requests.get(url, verify=False).text  # Doesn't work and is dangerous

# scamalytics didn't work so we used the curl method
with os.popen('curl ' + url) as pipe:
    html_text = pipe.read()
   
soup = BeautifulSoup()

In [45]:
lines

'<!-- init: 3.33 ms --><!-- getNmapResult: 0.24 ms --><!-- getProxies: 4.69 ms --><!-- reverseDNS: 0.14 ms --><!doctype html>\n<html ⚡>\n<head>\n\n    <meta charset="utf-8">\n    <title>52.252.130.105 ( Microsoft Corporation ) Fraud Risk</title>\n\n    <script async src="https://cdn.ampproject.org/v0.js"></script>\n    <script async custom-element="amp-form" src="https://cdn.ampproject.org/v0/amp-form-0.1.js"></script>\n    <script async custom-element="amp-analytics" src="https://cdn.ampproject.org/v0/amp-analytics-0.1.js"></script>\n\n    <link rel="canonical" href="https://scamalytics.com/ip/52.252.130.105">\n    <link rel="shortcut icon" href="https://scamalytics.com/wp-content/uploads/2016/06/icon_128.png" type="image/png">\n    <link rel="icon" href="https://scamalytics.com/wp-content/uploads/2016/06/icon_128.png" type="image/png">\n\n    <meta name="viewport" content="width=device-width,minimum-scale=1,initial-scale=1">\n\n    <style amp-boilerplate>body{-webkit-animation:-amp-s